In [ ]:
import sys
sys.path.insert(0, '../..')

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import edunn as nn
from edunn import utils

In [ ]:
np.set_printoptions(threshold=sys.maxsize)

# Capa Dropout

Dropout actúa como una técnica de _regularización_ que elimina o desactiva nodos en la propagación hacia adelante, lo que hace que la red sea menos propensa al sobreajuste al evitar que la red dependa demasiado de cualquier neurona individual. Esta capa no tiene parámetros.

* En la propagación hacia adelante, las entradas se establecen en cero con una probabilidad $p$, y de lo contrario se escalan por $\frac{1}{1-p}$.

  - La propagación hacia adelante durante el entrenamiento solo se utiliza para configurar la red para la propagación hacia atrás, donde la red se modifica realmente.
  
  - Para cada neurona individual en la capa, podemos decir que $x \sim B(1, p)$, ya que estamos considerando un solo "experimento" (la activación o desactivación de la neurona) con una probabilidad de éxito de $p$.

* En la propagación hacia atrás, los gradientes para las mismas unidades eliminadas se anulan; otros gradientes se escalan por el mismo factor de $\frac{1}{1-p}$.

  - Es decir, si un nodo fue eliminado por la capa Dropout, entonces su influencia (el gradiente) en los pesos salientes es también 0 (ya que $0 * w_i = 0$). En resumen, la propagación hacia atrás funciona como siempre.


> NOTA: tener en cuenta que durante la fase de prueba o validación, todas las neuronas están activas (es decir, no se aplica Dropout) para obtener una predicción basada en toda la red.

In [ ]:
np.random.seed(123)

din=10
batch_size=2

x = np.random.rand(batch_size,din)
                       
layer=nn.Dropout(p=0.5)

In [ ]:
x

In [ ]:
y = layer.forward(x)
y, y.shape

In [ ]:
# Define el gradiente de la salida
g = np.random.rand(*y.shape)

# Propaga el gradiente hacia atrás a través de la convolución
layer_grad = layer.backward(g)
layer_grad

In [ ]:
from edunn.model import Phase
layer.set_phase(Phase.Test)
y = layer.forward(x)
y, y.shape

## Comprobaciones con PyTorch

In [ ]:
import torch
import torch.nn as tnn

x = torch.from_numpy(x).to(torch.float)

# Definimos la capa Dropout
dropout = tnn.Dropout(p=0.5)

x.requires_grad = True

y_torch = dropout(x)
y_torch

In [ ]:
# Define el gradiente de la salida
g = torch.from_numpy(g).to(torch.double)

# Propaga el gradiente hacia atrás
x.grad = None  # Limpiamos los gradientes existentes
y_torch.backward(g)

# Imprime el gradiente de la imagen de entrada
print("Gradiente de la entrada (δE/δx):")
print(x.grad, x.grad.shape)

In [ ]:
dropout.eval()
y_torch = dropout(x)
y_torch